<a href="https://colab.research.google.com/github/Vineet2107/ML-for-Civil-Engineering-CE784A-/blob/main/Driver_Gaze_Zone_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
#imported necessary packages and libraries
import matplotlib.pyplot as plt
from skimage import io, color
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
import matplotlib.pyplot as plt
% matplotlib inline

In [76]:
#path of train and test data
train_path = '/content/drive/MyDrive/gaze_dataset/train'
test_path = '/content/drive/MyDrive/gaze_dataset/test'

In [77]:
#path of raveling, non revaling and test data
Centerstack_data = '/content/drive/MyDrive/gaze_dataset/train/Centerstack'
Forward_data = '/content/drive/MyDrive/gaze_dataset/train/Forward'
Left_wm_data = '/content/drive/MyDrive/gaze_dataset/train/Left_wing_mirror'
Rearview_data = '/content/drive/MyDrive/gaze_dataset/train/Rearview_mirror'
Right_wm_data = '/content/drive/MyDrive/gaze_dataset/train/Right_wing_mirror'
test_data = '/content/drive/MyDrive/gaze_dataset/test'

In [78]:
list_cs = os.listdir(Centerstack_data)
list_f = os.listdir(Forward_data)
list_lwm = os.listdir(Left_wm_data)
list_rv = os.listdir(Rearview_data)
list_rwm = os.listdir(Right_wm_data)
list_test = os.listdir(test_data)

In [79]:
data_dir = '/content/drive/MyDrive/gaze_dataset/train'


BATCH_SIZE = 32
# IMG_SIZE = (256, 256, 3)

train_dataset = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            )

Found 2505 files belonging to 5 classes.


In [80]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=8)

Found 2505 files belonging to 5 classes.
Using 501 files for validation.


In [81]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [82]:
num_classes = 5
model = tf.keras.Sequential([
  
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout((0.2)),
  tf.keras.layers.BatchNormalization(),
  
  tf.keras.layers.Flatten(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(num_classes,activation = 'softmax')
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


79/79 [==============================] - 26s 306ms/step - loss: 108.6317 - accuracy: 0.2351 - val_loss: 57.0316 - val_accuracy: 0.2974
Epoch 2/20
79/79 [==============================] - 21s 268ms/step - loss: 97.7923 - accuracy: 0.3066 - val_loss: 69.4945 - val_accuracy: 0.2435
Epoch 3/20
79/79 [==============================] - 21s 269ms/step - loss: 76.8583 - accuracy: 0.3545 - val_loss: 54.3750 - val_accuracy: 0.3832
Epoch 4/20
79/79 [==============================] - 21s 268ms/step - loss: 63.9398 - accuracy: 0.4335 - val_loss: 48.7296 - val_accuracy: 0.4531
Epoch 5/20
79/79 [==============================] - 21s 270ms/step - loss: 59.1971 - accuracy: 0.4922 - val_loss: 46.7392 - val_accuracy: 0.4671
Epoch 6/20
79/79 [==============================] - 21s 269ms/step - loss: 56.6149 - accuracy: 0.5102 - val_loss: 65.1914 - val_accuracy: 0.4731
Epoch 7/20
79/79 [==============================] - 21s 270ms/step - loss: 43.1697 - accuracy: 0.5860 - val_loss: 49.3231 - val_accuracy: 0.

In [98]:
from PIL import Image, ImageOps
data_dir = '/content/drive/MyDrive/gaze_dataset/test/'
test = ([(Image.open(data_dir+str(i+1)+".jpg")) for i in range(len(os.listdir(data_dir)))])

In [99]:
y_pred = model.predict(test)

ValueError: ignored

In [91]:
test_data = "/content/drive/MyDrive/gaze_dataset/test/"

In [92]:
train_dataset = tf.keras.utils.image_dataset_from_directory(test_data,
                                                            shuffle=False
                                                            )

Found 0 files belonging to 0 classes.


ValueError: ignored

In [73]:
#predicting on test data X_pred
y_pred = model.predict(val_ds)
y_output = []

In [ ]:
y_pred[0]

In [ ]:
result = np.where(y_pred[0] == np.amax(y_pred[0]))
print(result[0][0])

In [62]:
for i in range(len(y_pred)):
  result = np.where(y_pred[0] == np.amax(y_pred[0]))
  
  if(result == 0):
    y_output.append("Right_wing_mirror")
  if(result == 1):
    y_output.append("Rearview_mirror")
  if(result == 2):
    y_output.append("Left_wing_mirror")
  if(result == 3):
    y_output.append("Forward")
  else:
    y_output.append("Centerstack")

In [63]:
y_output[:10]

['Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack']

In [64]:
print(len(list_test))
print(len(y_output))

509
501


In [ ]:
for i in range(len(y_pred)):
  if(y_pred[i][0] > y_pred[i][1]):
    y_output.append("Non_raveling")
  else:
    y_output.append("Raveling")

In [ ]:
# stored non raveling image array in img_nr
img_cs = []
for i in range(len(list_cs)):
  img = io.imread(os.path.join(Centerstack_data,
                               list_cs[i]))
  img_cs.append(np.array(img))
  io.imshow(img)

img_f = []
for i in range(len(list_f)):
  img = io.imread(os.path.join(Forward_data,
                               list_f[i]))
  img_f.append(np.array(img))
  io.imshow(img)

img_lwm = []
for i in range(len(list_lwm)):
  img = io.imread(os.path.join(Left_wm_data,
                               list_lwm[i]))
  img_lwm.append(np.array(img))
  io.imshow(img)

img_rv = []
for i in range(len(list_rv)):
  img = io.imread(os.path.join(Rearview_data,
                               list_rv[i]))
  img_rv.append(np.array(img))
  io.imshow(img)

#stored raveling image array in img_r 
img_rwm = []
for i in range(len(list_rwm)):
  img = io.imread(os.path.join(Right_wm_data,
                               list_rwm[i]))
  img_rwm.append(np.array(img))
  io.imshow(img)

#stored test image array in img_test
img_test = []
for i in range(len(list_test)):
  img = io.imread(os.path.join(test_data,
                               list_test[i]))
  img_test.append(np.array(img))
  io.imshow(img)

In [ ]:
y_cs=np.empty(500)
y_cs.fill(0)

y_f=np.empty(500)
y_f.fill(1)

y_lwm=np.empty(500)
y_lwm.fill(2)

y_rv=np.empty(500)
y_rv.fill(3)

y_rwm=np.empty(500)
y_rwm.fill(4)

y = np.concatenate((y_cs, y_f, y_lwm, y_rv, y_rwm), axis = 0)

In [ ]:
perms = np.random.permutation(2500)

In [ ]:
X = np.concatenate((img_cs, img_f, img_lwm, img_rv, img_rwm), axis = 0)

In [ ]:
X = X[perms]
y = y[perms]